<a href="https://colab.research.google.com/github/maxiuboldi/challenge_ml/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Science Challenge Mercado Libre - BI Team

## Instalar dependencias

In [2]:
!pip install cherrypicker

  Created wheel for cherrypicker: filename=cherrypicker-0.3.1-cp36-none-any.whl size=13178 sha256=f4b0c07edd999adae32cff95009a305e509954cdcd4d9e94c689f1086ca3aa5f
  Stored in directory: /root/.cache/pip/wheels/ec/97/e2/90eef106c1f605a61836d9a5a8e8e47e96e1426e1ded7a10ee
Successfully built cherrypicker


## Importar librerias

In [36]:
import urllib.request
import urllib.parse
import json
import pandas as pd
import numpy as np
from math import ceil
from cherrypicker import CherryPicker
from time import sleep
import gc
from sklearn.preprocessing import OneHotEncoder

## Contantes

In [8]:
# API ML
URL_CATEGORIES = 'https://api.mercadolibre.com/sites/MLA/categories'
URL_CHILD_CATEGORIES = 'https://api.mercadolibre.com/categories/'
URL_SEARCH = 'https://api.mercadolibre.com/sites/MLA/search?'
LIMIT_CALLS = 1050 / 50
SLEEP = 0  # No encontré en la documentación límites, pero por las dudas...

# Modelo
SEED = 8888
K_FOLD = 5
TEST_SIZE = 0.20

# Opciones de Pandas

pd_options = {
'display.max_rows': None,
'display.max_columns': None,
'display.width': None,
'display.float_format': '{:.2f}'.format
}

[pd.set_option(option, setting) for option, setting in pd_options.items()]

[None, None, None, None]

## Descarga de categorías

In [5]:
print('Descargando categorías {}\n'.format(URL_CATEGORIES))
with urllib.request.urlopen(URL_CATEGORIES) as handler:
    categories = json.loads(handler.read())
sleep(SLEEP)

categories_list = []
for category in categories:
    url_categorie = URL_CHILD_CATEGORIES + category.get('id')
    print('Descargando sub categorías {}'.format(url_categorie))
    with urllib.request.urlopen(url_categorie) as handler:
        category_detail = json.loads(handler.read())
        for detalle in category_detail.get('children_categories'):
            categories_list.append(detalle)
    sleep(SLEEP)

categories_df = pd.DataFrame(categories_list)
categories_df.set_index('id', inplace=True)

print('\nListo')

Descargando categorías https://api.mercadolibre.com/sites/MLA/categories

Descargando sub categorías https://api.mercadolibre.com/categories/MLA5725
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1512
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1403
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1071
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1367
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1368
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1743
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1384
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1246
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1039
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1051
Descargando sub categorías https://api.mercadolibre.com/categories/MLA1648
Descargando sub categorías

## Muestreo de categorías adicionales y descarga de datos

In [6]:
print('\nMuestreando categorías')
categories_search_dict = categories_df[~categories_df['name'].str.lower().str.contains(
    r'tvs|smartphones')]['name'].sample(8, random_state=8888).to_dict()
categories_search_dict.update(categories_df[categories_df['name'].str.lower().str.contains(r'tvs|smartphones')]['name'])

print('Categorías seleccionadas')
for category, name in categories_search_dict.items():
    print(category, ': ', name)

print('Descargando resultados de búsqueda\n')
results_list = []
for category, name in categories_search_dict.items():
    print('Procesando categoría {}\n'.format(category))
    current_call = 0
    # La API sólo permite pasar un offset hasta 1.000, que devuelve un máximo de 1.050 artículos
    # con un máximo de 21 llamadas.
    for offset in range(0, 1050, 50):
        current_call += 1
        url_search_category = URL_SEARCH + urllib.parse.urlencode({'category': category, 'offset': offset})
        print('procesando {}'.format(url_search_category))
        with urllib.request.urlopen(url_search_category) as handler:
            search_result = json.loads(handler.read())
            total = search_result['paging'].get('total')
            if current_call == 1:
                print('Total de resultados: {}'.format(total))
            if total == 0:  # Por las dudas, hay categorías sin productos que pudieran salir del aleatorio
                break
            calls = ceil(total / 50)
            if current_call == 1:
                print('Total de llamadas necesarias: {}\n'.format(calls))
            results = search_result['results']
            for result in results:
                picker = CherryPicker(result)
                picker_dict = picker.flatten().get()
                picker_dict.update({'category_name': name})
                results_list.append(picker_dict)
            if current_call == calls:  # Si no hace falta llamar más, cortamos.
                break
            sleep(SLEEP)
    print('')

dataset = pd.DataFrame(results_list)
dataset.set_index('id', inplace=True)

del results_list, categories_search_dict, categories_list
gc.collect()

print('\nListo')


Muestreando categorías
Categorías seleccionadas
MLA1912 :  Otros
MLA1228 :  Otros
MLA385177 :  Climatización
MLA1700 :  Conectividad y Redes
MLA7841 :  Series de TV
MLA430687 :  Laptops y Accesorios
MLA7312 :  Maquinaria Agrícola
MLA404419 :  Insumos para Joyería
MLA1055 :  Celulares y Smartphones
MLA1002 :  TVs
Descargando resultados de búsqueda

Procesando categoría MLA1912

procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=0
Total de resultados: 7387
Total de llamadas necesarias: 148

procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=50
procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=100
procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=150
procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=200
procesando https://api.mercadolibre.com/sites/MLA/search?category=MLA1912&offset=250
procesando https://api.mercadolibre.com/sites/MLA/s

## Un poco de limpieza teniendo en cuenta el análisis preliminar

In [7]:
# Reemplazamos cadenas vacías por nan.
dataset.replace('', np.nan, inplace=True)

# Dropeamos columnas con todos valores vacíos
dataset.dropna(axis=1, how='all', inplace=True)

# Dropeamos columnas con todos valores constantes
dataset = dataset.loc[:, (dataset != dataset.iloc[0]).any()]

# Dropeamos las variables asociadas a URL's
dataset.drop(['seller_permalink', 'seller_eshop_eshop_logo_url', 'permalink', 
              'thumbnail', 'seller_car_dealer_logo', 'seller_home_image_url'],
             axis=1, inplace=True)

# Nos quedamos con las variables que parecieran relevantes para un análisis preliminar.
# TODO, revisar las variables de atributos para parsearlas correctamente.
dataset = dataset[['seller_registration_date', 'seller_car_dealer', 'seller_eshop_eshop_status_id',
                   'seller_seller_reputation_transactions_total', 'seller_seller_reputation_transactions_canceled',
                   'seller_seller_reputation_transactions_ratings_negative',
                   'seller_seller_reputation_transactions_ratings_positive',
                   'seller_seller_reputation_transactions_ratings_neutral',
                   'seller_seller_reputation_transactions_completed', 'seller_seller_reputation_power_seller_status',
                   'seller_seller_reputation_metrics_claims_rate', 'seller_seller_reputation_metrics_claims_value',
                   'seller_seller_reputation_metrics_delayed_handling_time_rate',
                   'seller_seller_reputation_metrics_delayed_handling_time_value',
                   'seller_seller_reputation_metrics_sales_completed',
                   'seller_seller_reputation_metrics_cancellations_rate',
                   'seller_seller_reputation_metrics_cancellations_value', 'seller_seller_reputation_level_id', 'price',
                   'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode', 'listing_type_id', 'condition',
                   'installments_quantity', 'installments_amount', 'installments_rate', 'shipping_free_shipping',
                   'shipping_mode', 'seller_address_state_name', 'original_price', 'official_store_id', 'domain_id',
                   'category_name', 'seller_seller_reputation_metrics_claims_excluded_real_rate',
                   'seller_seller_reputation_metrics_claims_excluded_real_value',
                   'seller_seller_reputation_metrics_delayed_handling_time_excluded_real_rate',
                   'seller_seller_reputation_metrics_delayed_handling_time_excluded_real_value',
                   'seller_seller_reputation_metrics_cancellations_excluded_real_rate',
                   'seller_seller_reputation_metrics_cancellations_excluded_real_value',
                   'seller_seller_reputation_real_level', 'seller_eshop_eshop_rubro_name']]

# Antigüedad del vendedor
dataset['seller_registration_date'] = pd.to_datetime(dataset['seller_registration_date']).dt.tz_localize(None).dt.normalize()
dataset['seller_year_aprox'] = round((pd.to_datetime('now').normalize() - dataset['seller_registration_date']).dt.days.fillna(0) / 360).astype('int32')

# Marca de tienda oficial?
dataset['flag_official_store'] = np.where(dataset['official_store_id'].fillna(0) == 0, 'No', 'Si')

# Borramos lo que ya no sirve
dataset.drop(['seller_registration_date', 'official_store_id'], axis=1, inplace=True)

## Separamos el target

In [24]:
data = dataset.loc[:, dataset.columns != 'sold_quantity']
target = dataset.loc[:, dataset.columns == 'sold_quantity'].rename(columns={'sold_quantity': 'target'})

## Removemos variables correlacionadas

In [35]:
data_num = data.select_dtypes(include='number').fillna(0)
data_cat = data.select_dtypes(exclude='number')

# Matriz de correlaciones
print('Forma antes: {}'.format(data_num.shape))
data_corr = data_num.corr().abs()
upper = data_corr.where(np.triu(np.ones(data_corr.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
data_num.drop(data_num[to_drop], axis=1, inplace=True)
print('Forma luego: {}'.format(data_num.shape))

Forma antes: (10441, 27)
Forma luego: (10441, 21)


## Codificamos variables categóricas

In [ ]:
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
gender = pd.DataFrame(enc.fit_transform(gender["legislador_genero"].values.reshape(-1, 1)),
                      columns=enc.get_feature_names(["legislador_genero"]), index=gender.index)